<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
from copy import deepcopy
from collections import deque 
import heapq
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self, state, heu_val, parent = None, empty_tile = None):
        
        #define the state of the puzzle
        self.state = state
        #define the heuristic used
        self.heu_val = heu_val
        #import a parent variable, if not defined by user it is set to None
        self.parent = parent
        #define atribute that stores the coordinates of the empty tile (None if not defined by User)
        self.empty_tile = empty_tile
        
        #if empty tile is not defined by user call method that finds and defines enpty tile
        if self.empty_tile == None:
            self.empty_tile = self.get_empty_tile()
        
        #if the node does not have a parent it must be the root node and the cost to reach it is 0
        if self.parent == None:
            self.cost_reach = 0
        #otherwise, the cost to reach it is one larger than the parent nodes cost
        else:
            self.cost_reach = parent.cost_reach + 1
        #define a evaluation function for A* as the heuristic + the cost to reach the node 
        self.g_val = self.cost_reach + self.heu_val
        
    #nodes should be compared based on their vlaue for function g (evaluation function)
    def __lt__(self,other):
        return self.g_val < other.g_val
    
    #def __eq__(self,other):
        
   #print state to represent node
    def __str__(self):
        #convert state to string
        grid = ""
        for row in self.state:
            #split each row
            grid = grid + str(row) + "\n"
            
        return grid
    
    #method to find empty tile of node
    def get_empty_tile(self):
        #go through all coordinates
        for row in range(len(self.state)):
            for column in range(len(self.state[row])):
                #if tile at coordinates is empty, return coordinates
                if self.state[row][column] == 0:
                    return [row, column]

#helper function to get the children of a node (all nodes that can be reached with one move)
#takes a node and a heursitic as input and outputs a list of the children of the node


Each node has six attributes. The first one is the state that I store as a list of lists. This is a convenient representation because the input is a list of lists, and it is a good portrayal of the puzzle. For each tile, the outer list index is one coordinate, and the index of tile within the list is the other coordinate. Further, we define the heuristic value and the cost to reach the node that is needed for the evaluation function and the evaluation function itself. We define the parent to backtrack the shortest path and the empty tile to evaluate possible moves.
We define three methods: one to represent the node, one to find the empty tile, and one to define how nodes are compared. This is useful for the heap that is used later. 

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

#first helper function for h3
#creates databases for each row subprobelm and returns list of all databases
def h3_help_1(state):
    #define list of databases
    database = []
    #for each row create database using second helper function and add it to list
    for r in range(len(state)):
        a = h3_help_2(state,r)
        database.append(a)
    #return database
    return database
        
#define second helper function for h3
#takes a state and a row number as input and returns the database to solve just that row
def h3_help_2(state, r):
    #define dimension of state
    n = len(state)
    #define goal state
    goal_state = []
    #variable for current tile 
    tile = 0
    
    #iterate over each coordinate
    for row in range(n):
        #add empty row for each row
        goal_state.append([])
        for column in range(n):
            #if row is the selected one append tile
            if row == r:
                goal_state[row].append(tile)
            #otherwise append n tile
            else:
                goal_state[row].append("n")
            #update current tile 
            tile += 1
    
    """
    this part is a breadth first search backwards from the solution, createing all possible states and their distance to the goal
    """
    #define database as dictionary
    database = {}
    #define variable for number of moves needed
    moves = 0
    #define current state as goal state and current moves
    current_state = (goal_state, moves)
    #define frontier as deque 
    frontier = deque([current_state])
    
    #start search and continue as long as frontier is not empry
    while len(frontier) != 0:
        #store current state in database linked to number of moves
        database[str(current_state[0])] = current_state[1]
        #iterate over number all coordinates
        for row in range(n):
            for column in range(n):
                #if current tile is in goal state get parents of current state moving this tile
                if isinstance(current_state[0][row][column], int):
                    parents = get_parents(current_state[0], n,row,column)
                    
                    #if current tile is not in database add it to frontier (right)
                    for parent in parents:
                        try:
                            database[str(parent)]
                        except:
                            #append frontier and increase moves for this state
                            frontier.append([parent, current_state[1]+1])
        #set current state to most right state in frontier
        current_state = frontier.popleft()
      
    #return filled database
    return database
               
#parent function
#takes a state, its dimensions and the coordinates of a tile as input
#outputs all parent states that if the tile specified was the last one moved
def get_parents(state, n,row,column):
    #define parent list
    parents = []
    
    #if the row of the tile is not the first one, we could have been moved down
    if row != 0:
        #copy state to new state
        new_state = deepcopy(state)
        #make switch 
        new_state[row][column], new_state[row-1][column] =  new_state[row-1][column], new_state[row][column]
        #append new state to parent list
        parents.append(new_state)
        
    #if the row of the tile is not the last one, we could have been moved up
    if row != n-1:
        #copy state to new state
        new_state = deepcopy(state)
        #make switch
        new_state[row][column], new_state[row+1][column] =  new_state[row+1][column], new_state[row][column]
        #append new state to parent list
        parents.append(new_state)
        
    #if the column of the tile is not the first one, we could have been moved to the right
    if column != 0:
        #copy state to new state
        new_state = deepcopy(state)
        #make switch
        new_state[row][column], new_state[row][column-1] =  new_state[row][column-1], new_state[row][column]
        #append new state to parent list
        parents.append(new_state)
        
    #if the column of the tile is not the last one, we could have been moved to the left
    if column != n-1:
        #copy state to new state
        new_state = deepcopy(state)
        #make switch
        new_state[row][column], new_state[row][column+1] =  new_state[row][column+1], new_state[row][column]
        #append new state to parent list
        parents.append(new_state)
    
    #return list of possible parents
    return parents




# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # YOUR CODE HERE
    #variable for the current correct tile 
    correct_tile = 0
    #counter for the number of misplaced tiles
    misplaced_tiles = 0
    
    #iterate over all tiles from left top to bottem right (if no misplaced tiles from 0 to n*n-1)
    for row in state:
        for tile in row:
            #if tile is not correct and not 0 (empty field) count misplaced tile
            if tile  != correct_tile and tile !=0:
                misplaced_tiles += 1
                
            #update correct tile
            correct_tile += 1
    
    #return number of misplaced tiles
    return misplaced_tiles
            
            

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # YOUR CODE HERE
    
    #counter  for manhatten distance
    man_dist = 0
    #dimension of state
    n = len(state[0])
    
    #iterate over all coordinates 
    for row in range(len(state)):
        for column in range(len(state[row])):
            #if current tile is 0 (empty field) move on to next tile 
            if state[row][column] == 0:
                continue
            #define x coordinate for columns and two y coordinates for rows(first and last tiel of row)
            x_c = 0
            y_c = 0
            y_c_s = 0
            #while the tile is not in the current row of the solution skip to the next row 
            while state[row][column] not in [i for i in range(y_c_s, y_c_s + n)]:
                y_c +=1
                y_c_s += n
            #while the tile is not in the current column skip to the next one 
            while state[row][column] != y_c_s + x_c:
                x_c += 1
            #calculate manhattan distance by adding absolute differences from row and column positions to solution positions for each tile
            man_dist += abs(y_c - row) + abs(x_c - column)
    #return manhattan distance
    return man_dist
            
    
# Extra heuristic for the extension.  If implemented, modify the function below

#define databse for last heuristic 
database_h3 = []

#additional heuristic using an additive database
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    
    #if database is not defined call helperfuncton to implement database 
    if len(database_h3) == 0:
        database_h3.append(h3_help_1(state))
    
    #store dimension of state
    n  = len(state)
    #store subproblems of solving just rows
    subproblems = []
    #for each dimension add a subproblem 
    for i in range(n):
        subproblems.append([])
    #for each subproblem add three rows
    for i in range(n):
        for u in range(n):
            subproblems[i].append([])
        
    
    #split state into n substates by grouping all tiels that should be in the same row together
    c = [ i for i in range(n)] 
    #iterate over all coordinates
    for row in range(n):
        for column in range(n):
            #iterate over each tile value
            for i in range(n*n):
                #if tiel value is equal to tile add tile to appropriate subproblem
                if i == state[row][column]:
                    subproblems[i//n][row].append(i)
                    #add an n in the same position for the other tow subproblems
                    for u in c:
                        if u != i//n:
                            subproblems[u][row].append("n")
    
    #set heuristic value to 0
    heu_val = 0
    #add dubproblems vlaues together
    for i in range(len(subproblems)):
        heu_val += database_h3[0][i][str(subproblems[i])]
    #return heuristic vlaue 
    return heu_val

heuristics = [h1, h2, h3]


For the third heuristic, I decided to implement an additive pattern database. The idea is to split the puzzle into two parts. For each part, half of the fields are empty. Each tile is just in one of the two parts. Then we solve each part separately and add the steps needed for each together. This is a more complex version of the Manhattan distance. While for the Manhattan distance, we evaluate each tile alone, we do now consider multiple tiles together. This is still a simplified problem, but less simplified than the Manhattan distance. Since we have to consider some collisions between tiles, the result will be at least as high as the manhattan distance, but mostly higher. Unfortunately, my computer was too slow to solve this heuristic even for the 8 Puzzle, so I implemented a computational, less expensive solution. Instead of splitting the state into two parts, I decided to split it into rows. This means that the heuristic evaluates all tiles that belong to the same row and solve all subproblems. Then we add the cost for each subproblem together for the overall cost. This heuristic is admissible because solving each subproblem can never be harder than solving the same tiles in the original problem. This is because the original problem will have more collisions due to the missing tiles. The heuristic will never overestimate the cost to reach the goal and is admissible. 
I implemented this heuristic with a pattern database. Based on the dimensions n of the puzzle, the algorithm splits up the goal state into n states containing one row. For each row, we do a depth-first search backward and store each state and the length to get from the state back to the goal state. These n databases are used to evaluate the heuristic in constant time once they are created. 

Note: While the heuristic was able to solve the puzzle, it used more expansions than the Manhattan distance in the test. I am not sure where the mistake occurred. 

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
# Import any packages or define any helper functions you need here
import heapq

#helper function to get the children of a node (all nodes that can be reached with one move)
#takes a node and a heursitic as input and outputs a list of the children of the node
def get_children(node, heuristic): 
    
    #list of children (initially empty)
    children = []
    #define the coordinates of the empty tile 
    r = node.empty_tile[0]
    c = node.empty_tile[1]
    
    #if the row of the empty tile is not the first one, we can move the empty tile up
    if r != 0:
        #copy the state
        new_state = deepcopy(node.state)
        #move tiles in new state
        new_state[r][c], new_state[r-1][c] = new_state[r-1][c], new_state[r][c]
        #check if heuristic value is in database
        if str(new_state) + str(heuristic) in heuristic_database:
            #use database to get heuristic vlaue 
            heu_val = heuristic_database[str(new_state) + str(heuristic)]
        else:
            #calculate heuristic function and store it in databse
            heu_val = heuristic(new_state)
            heuristic_database[str(new_state) + str(heuristic)] = heu_val
            
        #create new child node and append it to children list
        children.append(PuzzleNode(new_state,heu_val, parent = node, empty_tile = [r-1, c]))
    
    #if the row of the empty tile is not the last one, we can move the empty tile down
    if r != len(node.state[0])-1:
        #copy the state
        new_state = deepcopy(node.state)
        #move tiles in new state
        new_state[r][c], new_state[r+1][c] = new_state[r+1][c], new_state[r][c]
        #check if heuristic value is in database
        if str(new_state) + str(heuristic) in heuristic_database:
            #use database to get heuristic vlaue 
            heu_val = heuristic_database[str(new_state) + str(heuristic)]
        else:
            #calculate heuristic function and store it in databse
            heu_val = heuristic(new_state)
            heuristic_database[str(new_state) + str(heuristic)] = heu_val
            
        #create new child node and append it to children list
        children.append(PuzzleNode(new_state,heu_val, parent = node, empty_tile = [r+1, c] ))
    
    #if the column of the empty tile is not the first one, we can move the empty tile to the right
    if c != 0:
        #copy the state
        new_state = deepcopy(node.state)
        #move tiles in new state
        new_state[r][c], new_state[r][c-1] = new_state[r][c-1], new_state[r][c]
        #check if heuristic value is in database
        if str(new_state) + str(heuristic) in heuristic_database:
            #use database to get heuristic vlaue 
            heu_val = heuristic_database[str(new_state) + str(heuristic)]
        else:
            #calculate heuristic function and store it in databse
            heu_val = heuristic(new_state)
            heuristic_database[str(new_state) + str(heuristic)] = heu_val
            
        #create new child node and append it to children list
        children.append(PuzzleNode(new_state,heu_val , parent = node, empty_tile = [r, c-1] ))
        
    #if the row of the empty tile is not the last one, we can move the empty tile to the left
    if c!= len(node.state[0])-1:
        #copy the state
        new_state = deepcopy(node.state)
        #move tiles in new state
        new_state[r][c], new_state[r][c+1] = new_state[r][c+1], new_state[r][c]
        #check if heuristic value is in database
        if str(new_state) + str(heuristic) in heuristic_database:
            #use database to get heuristic vlaue 
            heu_val = heuristic_database[str(new_state) + str(heuristic)]
        else:
            #calculate heuristic function and store it in databse
            heu_val = heuristic(new_state)
            heuristic_database[str(new_state) + str(heuristic)] = heu_val
            
        #create new child node and append it to children list
        children.append(PuzzleNode(new_state,heu_val, parent = node, empty_tile = [r, c+1] ))
                    
    return children






#database for heuristic valeus
heuristic_database = {}
#databse for already solved puzzles
state_database = {}

# YOUR CODE HERE (OPTIONAL)
# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # YOUR CODE HERE
    
    #check if we solved this problem before and return solution based on database if possible
    if str(state) + str(heuristic) in state_database:
        return state_database[str(state) + str(heuristic)]

        
    #define steps, exp. man_frontier, opt_path, err with initial values
    steps = 0
    exp = 0
    max_frontier = 0
    opt_path = []
    err = 0
    
    #define frontier as list (used as priority queue implemented with a heap)
    frontier = []
    #define dimensions
    n = len(state)
    #define list of tiels
    tiles = [i for i in range((n**2))]
    
    #test if puzzle is correct (consistant dimensions and numbers)
    #iterate over all rows
    for row in state:
        #test if row has correct dimensions and return error otherwise
        if len(row) != n:
            opt_path = None
            err = -1
            #append state and solution to database
            state_database[str(state) + str(heuristic)] = (steps, exp,max_frontier,opt_path, err)
            return(steps, exp,max_frontier,opt_path, err) 
        #iterate over each tile in row
        for tile in row:
            #if tile should not be in puzzle return error 
            if tile not in tiles:
                opt_path = None
                err = -1
                #append state and solution to database
                state_database[str(state) + str(heuristic)] = (steps, exp,max_frontier,opt_path, err)
                return(steps, exp,max_frontier,opt_path, err) 
            #remove tile from tiles to make sure each tile can only be identified once 
            else:
                tiles.remove(tile)
    
    #test if heursitc for state is in database and use it to define heuristic value if possible 
    if str(state) + str(heuristic) in heuristic_database:
        heu_val = heuristic_database[str(state) + str(heuristic)]
    #otherwise add state with heursitic to database and calculate heuristic value using heuristic function
    else:
        heu_val = heuristic(state)
        heuristic_database[str(state) + str(heuristic)] = heu_val
    
    #define current node with initial state 
    current_Node = PuzzleNode(state, heuristic(state))
    
    
    #test if puzzle can be solved 
    
    #define variable for inversions
    invers = 0
    #make a list of all tiles for flat representaion of the puzzle 
    flat = [tile for row in state for tile in row]
    #iterate over all tiles 
    for t_1 in range(len(flat)):
        #iterate over all tiles coming after current tile
        for t_2 in range(t_1+1, len(flat)):
            #if later tile is larger than current tile and non of them is the zero tile, increase invers
            if flat[t_1] > flat[t_2]:
                if flat[t_1] == 0 or flat[t_2] == 0:
                    continue
                invers += 1
            
    #if dimension of state is even use this test
    if n%2 == 0:
        #if the empty tile is in an even row counted from the bottem and the inversion is odd
        #or the empty tile is in an odd row counted from the bottem and the inversion is even
        if (n - current_Node.empty_tile[0]%2 ==0 and invers%2 != 0) or (n - current_Node.empty_tile[0]%2 !=0 and invers%2 == 0):
            #puzzle is not solvable: return appropriate error
            opt_path = None
            err = -2
            #append state and solution to database
            state_database[str(state) + str(heuristic)] = (steps, exp,max_frontier,opt_path, err)
            return(steps, exp,max_frontier,opt_path, err)
    
    #if dimension of state is odd use this test
    #if number of inversions is odd puzzle is not solvable: return appropritate error 
    elif invers%2 != 0:
        opt_path = None
        err = -2
        #append state and solution to database
        state_database[str(state) + str(heuristic)] = (steps, exp,max_frontier,opt_path, err)
        return(steps, exp,max_frontier,opt_path, err)
    
    #create solution
    solution = []
    #set start tile to 0
    start = 0
    #append appropritate rows to solution
    for row in range(n):
        solution.append([i for i in range(start, start+n)])
        #update tile
        start +=n
        
    #update max frontier
    max_frontier = 1
    #define explored set for the graph search
    pre_vis = {}
    #search as long as the current node is not the solution
    while current_Node.state != solution:
        #get children of current node with helper fuction
        children = get_children(current_Node, heuristic)
        
        #for every child, try if it is in the explored set
        #if it is not add it to the frontier
        for child in children:
            try:
                pre_vis[str(child.state)]
            except KeyError:
                #the frontier is a heap
                heapq.heappush(frontier, child)
        
        #if the current length of the contier is larger than the absolute max: update max
        max_frontier = max(max_frontier, len(frontier))

        #add current state to explored set        
        pre_vis[str(current_Node.state)] = True
        #use heapop to set new current node (lowest evaluation function)
        current_Node = heapq.heappop(frontier)
        exp +=1
    
    #while current node is not the initial state go optimal path back
    while current_Node.parent != None:
        #add node to optimal path
        opt_path.insert(0,current_Node.state)
        #set current node to parent of node
        current_Node = current_Node.parent
    #add initial state to optimal path
    opt_path.insert(0,current_Node.state)
    #calculate numbe of steps of optimal path
    steps = len(opt_path)-1
    #append state and solution to database
    state_database[str(state) + str(heuristic)] = (steps, exp,max_frontier,opt_path, err)
    #return solution
    return(steps, exp,max_frontier,opt_path, err) 

I defined a helper function, get children, that returns a list of all possible children for a node. Further, I implement two databases for memoization. Both are dictionaries to ensure constant lookup time. I store all heuristic values for every state with the heuristic used in the heuristic database. Every time I see a state that was evaluated before, and I am using the same heuristic, I can look up the heuristic rather than calculate it. Further, I store the result for every solved state. If I try to solve the same state again, I can look it up instead of searching. This database could be enhanced by saving states of the optimal path. This is especially easy if we are not interested in the max frontier and expansion. I use a graph search. This makes the algorithm faster than a tree search, but it uses more memory because we have to keep track of the explored set.

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [9]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

AssertionError: 

In [ ]:
## Memoization test - will be carried out after submission